In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# plot image or not
is_plot = False

#### Data Loading

In [ ]:
from UltraMotionCapture import kps

vicon = kps.MarkerSet()
vicon.load_from_vicon('../UltraMotionCapture/data/6kmh_softbra_8markers_1.csv')
vicon.interp_field()

In [ ]:
from UltraMotionCapture import obj3d

o3_ls = obj3d.load_obj_series(
    folder='../UltraMotionCapture/data/6kmh_softbra_8markers_1/',
    start=0,
    stride=1,
    end=10,
    obj_type=obj3d.Obj3d_Deform
)

In [ ]:
from UltraMotionCapture import obj4d
from probreg import cpd

o4 = obj4d.Obj4d_Deform(
    fps=120,
    enable_rigid=False,
    enable_nonrigid=True,
)
o4.add_obj(*o3_ls)
o4.load_markerset('M8', vicon)

#### Displacement Field Estimation

The estimated non-rigid displacement field.

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('pythreejs')

if is_plot:
    o3_ls[0].show_deform(mode='nonrigid')

The estimated rigid transformation.

In [ ]:
o3_ls[0].show_deform(mode='rigid')

The difference of the ground-truth geometry (frame 2 object) and the deformed geometry under the estimated displacement.

In [ ]:
obj_deform = o3_ls[0].get_deform_obj3d()
obj_gt = o3_ls[1]

if is_plot:
    obj3d.Obj3d_Kps.show_diff(obj_deform, obj_gt)

#### Ultra-dense Motion Capture

In [ ]:
kps_source = vicon.get_time_coord(5/120)
o4.vkps_track(kps_source, frame_id=5)

In [ ]:
if is_plot:
    o4.show_deform_gif(output_folder='output', filename='obj4d_vkps', kps_names=('vkps', ))

Assemble the tracked virtual key points.

In [ ]:
vkps = o4.assemble_markerset(name='vkps')

In [ ]:
if is_plot:
    vkps.plot_frame(frame_id=0)

In [ ]:
diff = kps.MarkerSet.diff(vkps, vicon)